**Install Required Librairies**

In [1]:
!pip install openai datasets
!pip install gradio==4.10.0

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 223.7/223.7 kB 2.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 521.2/521.2 kB 9.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.0/75.0 kB 4.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.3/115.3 kB 10.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 10.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.9/76.9 kB 5.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 4.1 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
llmx 0.0.15a0 requires cohere, which is not installed.
llmx 0.0.15a0 requires tiktoken, which is not installed.
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 16.6/16.6 MB 30.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 93.1/93.1 kB 6.8

**Import gradio for UI**

In [2]:
import gradio

**Upload dataset from Hugging Face**


---



In [3]:
from datasets import load_dataset
ds = load_dataset("RachidAb02/FinancialData",split="train")

Extracting data files:   0%|          | 0/1 [00:00<?, ?it/s]

Generating train split: 0 examples [00:00, ? examples/s]

**Dataset Features**

In [4]:
len(ds)

132

In [5]:
ds[0]

{'question': "Qu'est-ce qu'un bilan comptable?",
 'answer': "Le bilan comptable est un document financier qui récapitule l'actif, le passif et les capitaux propres d'une entreprise à un moment donné."}

**format the data in the desired format**


In [6]:
ds_formatted = [
    {"messages": [
        {"role": "system", "content": "tu est un assistant financier des entreprises dans ses bilans comptables et ses situations financière.s'il te plait répond avec politesse.."},
        {'role': 'user', 'content': x["question"]},
        {'role': 'assistant', 'content': x["answer"]}]} for x in ds
]

In [7]:
ds_formatted[0]

{'messages': [{'role': 'system',
   'content': "tu est un assistant financier des entreprises dans ses bilans comptables et ses situations financière.s'il te plait répond avec politesse.."},
  {'role': 'user', 'content': "Qu'est-ce qu'un bilan comptable?"},
  {'role': 'assistant',
   'content': "Le bilan comptable est un document financier qui récapitule l'actif, le passif et les capitaux propres d'une entreprise à un moment donné."}]}

**Shuffle and split data into training and validation sets**

In [8]:
import random
random.shuffle(ds_formatted)

In [9]:
ds_train = ds_formatted[:100]
ds_val = ds_formatted[100:]

In [10]:
import json

with open('train.jsonl', 'w') as f:
    for line in ds_train:
        json.dump(line, f)
        f.write('\n')

with open('val.jsonl', 'w') as f:
    for line in ds_val:
        json.dump(line, f)
        f.write('\n')

**Get the API kety**

In [11]:
import openai

api_key ='api key'

In [12]:
# Upload training data
from pathlib import Path
from openai import OpenAI

client = OpenAI(api_key=api_key)

train =client.files.create(
    file=Path("train.jsonl"),
    purpose="fine-tune",
)
train

FileObject(id='file-OC8TNkvb8yDgIUEPKFj8oqLs', bytes=57459, created_at=1702825557, filename='train.jsonl', object='file', purpose='fine-tune', status='processed', status_details=None)

**Training set id**

In [13]:
train_id = train.id

In [14]:
# Upload validation data
val =client.files.create(
    file=Path("val.jsonl"),
    purpose="fine-tune",
)
val

FileObject(id='file-NEkKdssWQZli6QpW7u8o09L6', bytes=18744, created_at=1702825577, filename='val.jsonl', object='file', purpose='fine-tune', status='processed', status_details=None)

**validation set id**

In [15]:
val_id = val.id

**Finetuning the Model**

In [16]:
# Create a fine-tuned model
response = client.fine_tuning.jobs.create(
    training_file=train_id,
    validation_file=val_id,
    model="gpt-3.5-turbo"
    )
response

FineTuningJob(id='ftjob-X2qW7HW88NeGaIAYALrMejoA', created_at=1702825596, error=None, fine_tuned_model=None, finished_at=None, hyperparameters=Hyperparameters(n_epochs='auto', batch_size='auto', learning_rate_multiplier='auto'), model='gpt-3.5-turbo-0613', object='fine_tuning.job', organization_id='org-sRrbjKKDxD7iK8Xh7emL3Ayb', result_files=[], status='validating_files', trained_tokens=None, training_file='file-OC8TNkvb8yDgIUEPKFj8oqLs', validation_file='file-NEkKdssWQZli6QpW7u8o09L6')

**Get The model id**

In [17]:
model_id = "ft:gpt-3.5-turbo-0613:personal::8WMWDZD1"

**Test the output**

In [25]:
test_messages = [
    {"role": "system", "content": "tu es un assistant financier des entreprises en ce qui concerne le bilan comptable et le compte des produits et des charges , s'il te plait genere des tableux pour ces taches. repond avec politesse."},
    {"role": "user", "content": "tu peux m'aider a schematiser des operations dans des tableaux ?"}
  ]

In [19]:
completion = client.chat.completions.create(
  model=model_id,
  messages=test_messages
)

In [20]:
print(completion.choices[0].message)

ChatCompletionMessage(content='Bien sûr! Les tableaux peuvent être utiles pour représenter des opérations financières. Que souhaitez-vous représenter exactement?', role='assistant', function_call=None, tool_calls=None)


**Create User interface with gradio**

In [28]:
def generate_openai_response(prompt):
    completions =client.chat.completions.create(
        model="gpt-3.5-turbo",
        messages=[
        {"role": "user", "content": prompt}
    ]
    )
    message = completions.choices[0].message.content
    return message.strip()

In [29]:
def chatbot(Prompt, conversation_history=[]):
    response_text = generate_openai_response(Prompt)
    conversation_history.append((Prompt, response_text))
    return conversation_history, conversation_history

**User Interface**

In [ ]:
gradio.Interface(fn = chatbot,
             inputs = ["text",'state'],
             outputs = ["chatbot",'state'],
             title="Khabiri",
             description="Bienvenue A Khabiri").launch(debug = True)

Setting queue=True in a Colab notebook requires sharing enabled. Setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. This cell will run indefinitely so that you can see errors and logs. To turn off, set debug=False in launch().
Running on public URL: https://a8961a5527348a510a.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from Terminal to deploy to Spaces (https://huggingface.co/spaces)
